In [5]:
import numpy as np
from scipy.spatial import distance_matrix

def get_centroid(detections, transposed = True):
    """
    :param detections: N x 3/4 or 3/4 x N
    :return: 3 x 1
    """
    if (transposed):  # N x 4
        return np.mean(detections[:, :3], 0, keepdims=True)
    else: # 4 x N
        return np.mean(detections[:3, :], 1, keepdims=True)

def get_mean_distance(detections, transposed = False):
    """
    :param detections: 3 x N
    :return:
    """

    if (transposed): # 3 x N
        pass
    else: # N x 3
        detections = detections.transpose() # N x 3

    if detections.shape[1] == 4:
        detections = detections[:, :3] # N x 3
    pair_wise_distance = []
    for i in range(detections.shape[0]):
        for j in range(i + 1, detections.shape[0]):
            pair_wise_distance.append(np.linalg.norm(detections[i] - detections[j]))
    return np.average(pair_wise_distance)

def get_error(moving_landmarks, fixed_landmarks):
    """
    :param moving_landmarks: 3 x N
    :param fixed_landmarks: 3 x N
    :return:
    """
    if (moving_landmarks is not None or fixed_landmarks is not None):
        residual = moving_landmarks - fixed_landmarks
        return np.mean(np.linalg.norm(residual, axis=0)) # axis= 0 means norm is taken along axis = 0
    else:
        return None

def get_affine_transform(moving, fixed, with_ones=False):
    """
    :param moving: point cloud 3 x N
    :param fixed: point cloud 3 x N
    :param with_ones: False
    :return: 4 x 4 affine matrix
    """
    if (with_ones):
        pass
    else:
        extra_one_row = np.ones((1, moving.shape[1]))
        moving = np.vstack((moving, extra_one_row))
        fixed = np.vstack((fixed, extra_one_row))
    return np.matmul(fixed, np.linalg.pinv(moving))


# http://www.sci.utah.edu/~shireen/pdfs/tutorials/Elhabian_ICP09.pdf
def get_similar_transform(moving, fixed):
    """
    :param moving: point cloud 3 x N
    :param fixed: point cloud 3 x N
    :return: s (1) , R (3 x 3) , t (3 x 1)
    """
    com_target = np.mean(fixed, 1, keepdims=True)  # 3 x 1
    com_source = np.mean(moving, 1, keepdims=True)  # 3 x 1

    # eliminate translation
    Yprime = fixed[:3, :] - com_target[:3, :]
    Pprime = moving[:3, :] - com_source[:3, :]

    # use quarternions
    Px = Pprime[0, :]
    Py = Pprime[1, :]
    Pz = Pprime[2, :]

    Yx = Yprime[0, :]
    Yy = Yprime[1, :]
    Yz = Yprime[2, :]

    Sxx = np.sum(Yx * Px)
    Sxy = np.sum(Px * Yy)
    Sxz = np.sum(Px * Yz)

    Syx = np.sum(Py * Yx)
    Syy = np.sum(Py * Yy)
    Syz = np.sum(Py * Yz)

    Szx = np.sum(Pz * Yx)
    Szy = np.sum(Pz * Yy)
    Szz = np.sum(Pz * Yz)

    Nmatrix = [[Sxx + Syy + Szz, Syz - Szy, -Sxz + Szx, Sxy - Syx],
               [-Szy + Syz, Sxx - Szz - Syy, Sxy + Syx, Sxz + Szx],
               [Szx - Sxz, Syx + Sxy, Syy - Szz - Sxx, Syz + Szy],
               [-Syx + Sxy, Szx + Sxz, Szy + Syz, Szz - Syy - Sxx]]

    [V, D] = np.linalg.eig(Nmatrix)  # TODO values, vectors
    # optimal quarternion vector corresponds to the largest eigen value
    idx = V.argsort()[::-1]
    V = V[idx]
    D = D[:, idx]

    q = D[0]  # first value is largest value!
    q0 = q[0]
    q1 = q[1]
    q2 = q[2]
    q3 = q[3]

    Qbar = [[q0, -q1, -q2, -q3],
            [q1, q0, q3, -q2],
            [q2, -q3, q0, q1],
            [q3, q2, -q1, q0]]

    Q = [[q0, -q1, -q2, -q3],
         [q1, q0, -q3, q2],
         [q2, q3, q0, -q1],
         [q3, -q2, q1, q0]]

    R = np.matmul(np.transpose(Qbar), Q)
    R = R[1:, 1:]

    # compute scaling
    Sp = 0
    D = 0

    for i in range(Yprime.shape[1]):
        D += np.matmul(np.transpose(Yprime[:, i]), Yprime[:, i])
        Sp += np.matmul(np.transpose(Pprime[:, i]), Pprime[:, i])

    s = np.sqrt(D / Sp)
    t = com_target[:3, :] - s * np.matmul(R, com_source[:3, :])
    A = np.zeros((4, 4))
    A[:3, :3] = s * R
    A[:3, 3:4] = t
    A[3, 3] = 1
    return A

def perform_icp(moving, fixed, icp_iterations = 50, transform='Affine'):
    if moving.shape[0] == 4:  # 4 x N
        moving = moving[:3, :] # 3 x N
    if fixed.shape[0] == 4:  # 4 x N
        fixed = fixed[:3, :] # 3 x N

    A_icp = np.identity(4)
    for i in range(icp_iterations):
        cost_matrix=distance_matrix(moving.transpose(), fixed.transpose()) # (N x 3, N x 3)
        i2=np.argmin(cost_matrix, 1)
        if(transform=='Affine'):
            A_est=get_affine_transform(moving, fixed[:, i2])
        elif(transform=='similar'):
            A_est = get_similar_transform(moving, fixed[:, i2])
        elif(transform=='rigid'):
            A_est = get_rigid_transform(moving, fixed[:, i2])
        moving = apply_affine_transform(moving, A_est)
        print("Residual at iteration {} is {}".format(str(i), get_error(moving, fixed[:, i2])))
        A_icp=np.matmul(A_est, A_icp)
    return A_icp
    

The below fails because the number of points are not the same.

In [8]:
points = np.loadtxt('/Users/kmarsh/Code/kmarsh/autogeoreferencer/data/OH-Champaign-72-01.csv', delimiter=',', usecols=(0, 1))
intersections = np.loadtxt('/Users/kmarsh/Code/kmarsh/autogeoreferencer/data/39021-intersections-3857.psv', delimiter='|')

perform_icp(points, intersections, icp_iterations=50)

ValueError: x contains 24-dimensional vectors but y contains 502-dimensional vectors